# 웹에서 자동으로 E-book 받기

- [Ebook 굉장히 많다!](http://ebooks.shahed.biz/)
- 음.. 내가 했을때는 잘 됐는데 자동다운로드 하는 사람들이 많아서 그런가 **막아놨네** -ㅁ-(2014.07 정도)
- 지금은 id/pw 입력해야 된다. 그래서 지금 이 소스를 돌리면 바로 종료됨..
- id/pw 자동 입력해서 다운로드 받을 수 있게 업그레이드 해야 됨. 근데 귀찮음. 그건 **독자의 몫**으로 남김

![ebooks](files/images/auto_download_site.png)

<img src="files/images/auto_download.png" width=600px />


## 만들게 된 계기

- 하나씩 클릭하면서 다운 받는건 노가다니까..
- 폴더도 자동으로 생성
- 재귀적으로 폴더, 파일 분류

## 다양한 방법 가능

1. 손으로..
2. 툴의 도움(Download master 같은)
3. [Download master - Chrome Extension](https://chrome.google.com/webstore/detail/download-master/mcceagdollnkjlogmdckgjakjapmkdjf): 이것도 좋다. 링크들만 추출해주고 다운받을 수 있게 도와준다. 그런데 서브 디렉토리까지 있는건 약간의 노가다가 필요하다. 최종결론: 이건 반노가다
4. 직접 짜본다. **스크립트 작성**(이걸로 선택)

## 다운로드 받는 Logic

1. Base_url을 입력한다.
2. requests를 해서 reponse를 받아온다.
3. get_links에 response를 넘겨서 links를 추출한다.(상대경로 호환해줘야 된다.)
4. 디렉토리라면 한 번 더 들어가고 파일이면 그 파일을 받아온다.
5. 파일을 저장할 때 디렉토리가 없으면 생성해준다.
6. 파일을 저장한다.

## 최종 소스 코드 

In [2]:
# -*- coding: utf-8

import requests
import re
import os
import urllib2
from bs4 import BeautifulSoup

def is_dir(string):
    """confirm ended words with /"""
    if re.search(r'.*?\/$', string):
        return True
    return False

def is_file(string):
    """confirm ended words without /"""
    if re.search(r'[^\/]$', string):
        return True
    return False

def replace_os_character(string):
    """misunderstand chracters in OS change to '_'."""
    # except /
    string = re.sub(r'\\|\?|\:|\*|\"|\>|\<|\|', '_', string)
    return string

def get_links(soup, base_url):
    """get links"""
    links = []
    for link in soup.find_all('a'):
        link = link.get('href').strip()
        if re.search(r'^[A-Za-z0-9]', link) and \
                not 'www.shahed.biz' in link:
            links.append(os.path.join(base_url, link))
    return links

def do_file_work(link, save_base_path='/tmp'):
    """file download"""
    try:
        print 'downloading...', link
        response = requests.get(link)
        filename = re.sub(r'http://ebooks.shahed.biz/', '', link)
        fullpath = os.path.join(save_base_path, filename)
        fullpath = urllib2.unquote(fullpath)
        fullpath = replace_os_character(fullpath)
        path, filename = os.path.split(fullpath)
        write_file(fullpath, path, filename, response)
    except Exception, e:
        print e

def write_file(fullpath, path, filename, response):
    """make dir and write file"""
    if not os.path.exists(path):
        os.makedirs(path)

    try:
        with open(fullpath, 'wb') as f:
            f.write(response.content)
            print '[+] wrote', filename
    except Exception, e:
        print e

def search(url):
    """main module"""
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.content)
        links = get_links(soup, url)
        for link in links:
            # link = os.path.join(url, link)
            if is_dir(link):
                print '+', link
                search(link)
            else:
                print '-', link
                do_file_work(link)
            print ''
    except Exception as e:
        print e

base_url = 'http://ebooks.shahed.biz/'

if __name__ == '__main__':
    search(base_url)